In [4]:
from helper import get_input

INPUTS = get_input(day=2).strip()
INPUTS

'5959566378-5959623425,946263-1041590,7777713106-7777870316,35289387-35394603,400-605,9398763-9592164,74280544-74442206,85684682-85865536,90493-179243,202820-342465,872920-935940,76905692-76973065,822774704-822842541,642605-677786,3759067960-3759239836,1284-3164,755464-833196,52-128,3-14,30481-55388,844722790-844967944,83826709-83860070,9595933151-9595993435,4216-9667,529939-579900,1077949-1151438,394508-486310,794-1154,10159-17642,5471119-5683923,16-36,17797-29079,187-382'

Decided to build a dataclass for the input ranges, producing a `start`, `end`, and an interator that yields the numbers, inclusive.

In [5]:
from dataclasses import dataclass


@dataclass
class RangeThing:
    start: int
    end: int

    def __iter__(self):
        yield from range(self.start, self.end + 1)

    @classmethod
    def from_input_line(cls, line: str):
        return cls(*map(int, line.split("-")))


RANGES = [RangeThing.from_input_line(x) for x in INPUTS.split(",")]
RANGES[:5]

[RangeThing(start=5959566378, end=5959623425),
 RangeThing(start=946263, end=1041590),
 RangeThing(start=7777713106, end=7777870316),
 RangeThing(start=35289387, end=35394603),
 RangeThing(start=400, end=605)]

## Part 1
### Insights

I need to start from a number greater than the `start` that also has an even number of digits.
Then, I can check if the first half of digits *duplicated* results in a number that is less than the `end` number.
This can be achieved lexicographically, so long as the 'test' number has the same number of digits as the `end` itself.

NOTE: If the `start` and `end` have the same number of digits, and both have an *odd* number of digits, then there are no invalid IDs.

In [6]:
TEST_INPUT = (
    "11-22,95-115,998-1012,1188511880-1188511890,222220-222224,"
    "1698522-1698528,446443-446449,38593856-38593862,565653-565659,"
    "824824821-824824827,2121212118-2121212124"
)
TEST_RANGES = [RangeThing.from_input_line(x) for x in TEST_INPUT.split(",")]
TEST_RANGES

[RangeThing(start=11, end=22),
 RangeThing(start=95, end=115),
 RangeThing(start=998, end=1012),
 RangeThing(start=1188511880, end=1188511890),
 RangeThing(start=222220, end=222224),
 RangeThing(start=1698522, end=1698528),
 RangeThing(start=446443, end=446449),
 RangeThing(start=38593856, end=38593862),
 RangeThing(start=565653, end=565659),
 RangeThing(start=824824821, end=824824827),
 RangeThing(start=2121212118, end=2121212124)]

I'll add some utility functions that I can use in the part1 solution to chop numbers up and form new ones.

In [7]:
def chop_val(val: int) -> str:
    x = str(val)
    if len(x) % 2:
        raise ValueError("Not an even-length number")
    return x[: len(x) // 2]


def get_val_from_half(val: str) -> int:
    return int(val * 2)


def increment_str_int_by_1(val: str) -> str:
    return str(int(val) + 1)

In [8]:
def part1(inputs: list[RangeThing]) -> int:
    result = 0
    for thing in inputs:
        curr_val = thing.start
        if (len_val := len(str(curr_val))) % 2:
            curr_val = int("1" + "0" * len_val)

        curr_val_halfstr = chop_val(curr_val)
        curr_val = get_val_from_half(curr_val_halfstr)
        while curr_val < thing.start:
            # Say the value is 1122
            # The half string "11" results in "1111",
            # but this is lower than the start value.
            curr_val_halfstr = increment_str_int_by_1(curr_val_halfstr)
            curr_val = get_val_from_half(curr_val_halfstr)

        while curr_val <= thing.end:
            # print("INVALID ID", curr_val, "IN", thing)
            result += curr_val
            curr_val_halfstr = increment_str_int_by_1(curr_val_halfstr)
            curr_val = get_val_from_half(curr_val_halfstr)
    return result


print("EXPECTED", 1227775554)
print("TEST    ", part1(inputs=TEST_RANGES))
print("RESULT  ", part1(inputs=RANGES))

EXPECTED 1227775554
TEST     1227775554
RESULT   19605500130


## Part 2

I *was* attempting this one in a similar fashion, chopping numbers up using substrings,
duplicating those parts up to the length of the original value,
checking if that value is inside the range, etc.

I ran out of patience after a couple days not working on this,
so I went with the brute force approach:
iterate over numbers in each range, check if it's valid,
and simply add them up.

In [9]:
def part2(inputs: list[RangeThing]) -> int:
    result = 0

    def is_invalid(val: int):
        val_str = str(val)
        len_str = len(val_str)
        for sublen in range(1, (len_str // 2) + 1):
            if len_str % sublen:
                # Not divisible by this substring length: skip
                continue

            substr = val_str[:sublen]
            if not list(filter(None, val_str.split(substr))):
                return True
        return False

    for thing in inputs:
        for num in thing:
            if is_invalid(num):
                result += num
    return result


print("EXPECTED", 4174379265)
print("TEST    ", part2(inputs=TEST_RANGES))
print("RESULT  ", part2(inputs=RANGES))

EXPECTED 4174379265
TEST     4174379265
RESULT   36862281418


While it's not elegant, and there's certainly faster ways to achieve this, my local hardware is pretty beefy as it is; so waiting a whole 1.4s for a result didn't bother me much.